### Importing libraries

In [8]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

### loading dataset

In [2]:
path = "Dataset\song data"
data1 = pd.read_csv(os.path.join(path,"song_data.csv"))
data1.drop_duplicates(inplace=True)
print(data1.shape)
data1.head()

(999502, 5)


,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [3]:
data2 = pd.read_csv(os.path.join(path,"app_data.csv"))
data2.drop_duplicates(inplace=True)
print(data2.shape)
data2.head()

(2000000, 3)


,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


### Exploring  Data

In [4]:
user_count = data2.groupby(["song_id"]).agg({'user_id':'count'}).reset_index()
user_count.sort_values(by="user_id", ascending=False)

,song_id,user_id
2220,SOFRQTD12A81C233C0,8277
317,SOAUWYT12A81C206F1,7032
352,SOAXGDH12A8C13F8A1,6949
614,SOBONKR12A58A7A7E0,6412
7416,SOSXLTC12AF72A7F54,6145
...,...,...
8747,SOWNLZF12A58A79811,51
4492,SOLIGVL12AB017DBAE,51
622,SOBPGWB12A6D4F7EF3,50
9638,SOYYBJJ12AB017E9FD,48


In [5]:
song_data = pd.merge(data1, user_count, on="song_id")
print(song_data.shape)
song_data.head()

(10131, 6)


,song_id,title,release,artist_name,year,user_id
0,SOSZNRJ12A8AE46E38,Anyone Else But You,Juno - Music From The Motion Picture,Michael Cera & Ellen Page,2007,118
1,SOGKGLB12A81C22AFA,Drunk and Hot Girls,Graduation,Kanye West / Mos Def,2007,213
2,SOWZDNH12A6D4F7237,Until The Day I Die (Album Version),Page Avenue,Story Of The Year,2003,71
3,SOQBGZD12AB0184341,Rewind (Demo),Riot!,Paramore,0,97
4,SOTPWHK12A8AE46DC8,Never Knew Love,Love Songs,Rick Astley,1991,72


In [6]:
song_data.rename(columns={'artist_name':'artist', 'release':'album', 'user_id':'user_count'}, inplace=True)
song_data.columns

Index(['song_id', 'title', 'album', 'artist', 'year', 'user_count'], dtype='object')

### Creating Database

In [1]:
import sqlite3
conn = sqlite3.connect("song.db")
cur = conn.cursor()

In [8]:
cur.execute(
            '''CREATE TABLE IF NOT EXISTS song_data (song_id text, title text, release text, artist text, 
            year integer, user_count integer, '');'''
            )
conn.commit()

In [9]:
#When adding data for the first time or want to replace the complete table
song_data.to_sql('song_data', conn, if_exists='replace', index = False)

In [10]:
cur.execute(
            '''CREATE TABLE IF NOT EXISTS app_data (user_id text, song_id text, listen_count integer);'''
            )
conn.commit()

In [11]:
#When adding data for the first time or want to replace the complete table
data2.to_sql('app_data', conn, if_exists='replace', index = False)

## Item Item Collaborative Similarity

In [4]:
query = cur.execute("Select * from app_data;")
app_data = cur.fetchall()

In [5]:
data = pd.DataFrame(app_data)
data.rename(columns={0:"user_id", 1:'song_id', 2:'listen_count'}, inplace=True)
data.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [263]:
# The listen count is not considered here only if the user has listenn to the song or not
values = []
for i in data['listen_count']:
    if i > 0:
        values.append(1)
    else:
        values.append(0)

In [264]:
data['values'] = values
data.head()

,user_id,song_id,listen_count,values
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,1


In [265]:
def get_users_by_song_id(song_id):
    return  set(data['user_id'][data['song_id']==song_id])

In [266]:
users_of_target_song = get_users_by_song_id(target_song_id)
users_of_target_song

{'0e2f6624a9205bbfec266299bdfcf01c1fbef6d9',
 '3ff7a31452eeabd7a4e07f0d243c674e3d0adf46',
 '43c6db7ff4608324722a10cc95bd1c5c69ab7952',
 '6db92ef7b67cb86dc18adb7a3ffd81bfa1c2a0da',
 '8d705342bc1e2c5d6e51cacdaf61325e4020b4a9',
 'be0db87ab60e3a2de60354f34d9e37341b65eb1e'}

In [267]:
app_table = pd.pivot_table(data, index='user_id', columns='song_id', values='values', fill_value=0)
app_table.head()

song_id,SOAAAGQ12A8C1420C8,SOAACPJ12A81C21360,SOAACSG12AB018DC80,SOAAEJI12AB0188AB5,SOAAFAC12A67ADF7EB,SOAAFYH12A8C13717A,SOAAJMQ12A6D4F7D17,SOAAKPM12A58A77210,SOAALWN12A6D4F7FDA,SOAAMOW12AB018149B,...,SOZZKPR12A6D4F8147,SOZZLTY12A67AE0AD0,SOZZLZN12A8AE48D6D,SOZZRHE12A6702165F,SOZZTCU12AB0182C58,SOZZTNF12A8C139916,SOZZVWB12AB0189C30,SOZZWZV12A67AE140F,SOZZYAO12A6701FF36,SOZZZPV12A8C1444B5
user_id,,,,,,,,,,,,,,,,,,,,,
0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000b474f815bcff17a4bc9ce5324f9352dafe07d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000ebc858861aca26bac9b49f650ed424cf882fc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001f22c638730aed5659034c447d3cf0e658898e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
002543003041db1d049206b09426d5cdffc0f451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [359]:
target_song_id = "SOAACPJ12A81C21360"

In [360]:
user_by_song_id = app_table[target_song_id]

In [361]:
start = time()
user_by_song_id = app_table[target_song_id]
similar_to_target_song = app_table.corrwith(user_by_song_id)
similar_to_target_song.sort_values(ascending=False, inplace=True)
end = time()
print(end-start)
similar_to_target_song

4.587190389633179


song_id
SOAACPJ12A81C21360    1.000000
SOYDXEN12AF729D542    0.257503
SOFHIQQ12AB017ACE1    0.223573
SOTXONM12A8C13AAA6    0.222239
SOATLAK12A8C13F422    0.222239
                        ...   
SOTWNDJ12A8C143984   -0.010539
SODJWHY12A8C142CCE   -0.011531
SOLFXKT12AB017E3E0   -0.011785
SONYKOW12AB01849C9   -0.013105
SOFRQTD12A81C233C0   -0.015887
Length: 9997, dtype: float64

In [337]:
query = cur.execute("Select song_id, title, album, artist from song_data;")
song_data = cur.fetchall()

In [338]:
song_data = pd.DataFrame(song_data)
song_data.rename(columns={0:'song_id', 1:'title', 2:'album', 3:'artist'}, inplace=True)
song_data

,song_id,title,album,artist
0,SOSZNRJ12A8AE46E38,Anyone Else But You,Juno - Music From The Motion Picture,Michael Cera & Ellen Page
1,SOGKGLB12A81C22AFA,Drunk and Hot Girls,Graduation,Kanye West / Mos Def
2,SOWZDNH12A6D4F7237,Until The Day I Die (Album Version),Page Avenue,Story Of The Year
3,SOQBGZD12AB0184341,Rewind (Demo),Riot!,Paramore
4,SOTPWHK12A8AE46DC8,Never Knew Love,Love Songs,Rick Astley
...,...,...,...,...
10126,SOKGVJH12A58A77920,Grown So Ugly,Rubber Factory,The Black Keys
10127,SOINBEP12AB017FEE1,Island,Rules,The Whitest Boy Alive
10128,SOETNKM12A8AE47EEA,House By the Sea (Album),The Shepherd's Dog,Iron And Wine
10129,SOKATLY12A8C132FE2,FU-GEE-LA,The Score,Fugees


In [362]:
list(similar_to_target_song.index[1:13])

['SOYDXEN12AF729D542',
 'SOFHIQQ12AB017ACE1',
 'SOTXONM12A8C13AAA6',
 'SOATLAK12A8C13F422',
 'SOJFARO12AF72A709A',
 'SOIQKPT12A8C140753',
 'SOBAAAB12A6D4F95A5',
 'SOPGBMT12A8C142E7F',
 'SOYPBSP12A6D4F8EFC',
 'SOUGLCH12A8C140306',
 'SORPRMD12A6D4F9C5C',
 'SOTPQFM12AB017AC9E']

In [363]:
song_data[song_data['song_id']=='SOVMHDH12A58A76EDF']

,song_id,title,album,artist
5824,SOVMHDH12A58A76EDF,Street Dreams,Greatest Hits,Nas featuring R. Kelly


In [355]:
similar_matrix = dict()

In [356]:
similar_matrix[target_song_id] = list(similar_to_target_song.index[1:12])

In [357]:
similar_matrix

{'SOAAAGQ12A8C1420C8': ['SOOSVUB12A8C13B13F',
  'SOTQMGD12A81C2289C',
  'SOGUVMD12AB0183C82',
  'SOWONQS12A6D4FC79A',
  'SOVJUYC12AB018079C',
  'SONSPHT12A81C2289A',
  'SOSCDWE12AB01823C4',
  'SODMBJL12A8C13EB1A',
  'SOGQANL12A6701E099',
  'SOALDLA12A6D4F8657',
  'SOVMHDH12A58A76EDF']}

In [358]:
similar_matrix = pd.DataFrame(similar_matrix).T

In [364]:
similar_matrix.loc[target_song_id] = list(similar_to_target_song.index[1:12])

In [365]:
similar_matrix

,0,1,2,3,4,5,6,7,8,9,10
SOAAAGQ12A8C1420C8,SOOSVUB12A8C13B13F,SOTQMGD12A81C2289C,SOGUVMD12AB0183C82,SOWONQS12A6D4FC79A,SOVJUYC12AB018079C,SONSPHT12A81C2289A,SOSCDWE12AB01823C4,SODMBJL12A8C13EB1A,SOGQANL12A6701E099,SOALDLA12A6D4F8657,SOVMHDH12A58A76EDF
SOAACPJ12A81C21360,SOYDXEN12AF729D542,SOFHIQQ12AB017ACE1,SOTXONM12A8C13AAA6,SOATLAK12A8C13F422,SOJFARO12AF72A709A,SOIQKPT12A8C140753,SOBAAAB12A6D4F95A5,SOPGBMT12A8C142E7F,SOYPBSP12A6D4F8EFC,SOUGLCH12A8C140306,SORPRMD12A6D4F9C5C


## Co-occurence Matrix

In [2]:
import sqlite3
conn = sqlite3.connect("song.db")
cur = conn.cursor()

In [3]:
query = cur.execute("Select * from app_data;")
app_data = cur.fetchall()

In [4]:
data = pd.DataFrame(app_data)
data.rename(columns={0:"user_id", 1:'song_id', 2:'listen_count'}, inplace=True)
data.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [8]:
all_users_of_song = dict()
for song_id in tqdm(all_songs):
    all_users_of_song[song_id] = get_users_by_song_id(song_id)

100%|████████████████████████████████████| 10000/10000 [52:38<00:00,  3.17it/s]


In [9]:
import pickle
pickle.dump(all_users_of_song, open("all_users_by_song", "wb"))

In [5]:
all_users_of_song = pickle.load(open("all_users_by_song", "rb"))

In [39]:
all_songs = all_users_of_song.keys()
print(len(all_songs))
print(all_songs)

10000
dict_keys(['SOAKIMP12A8C130995', 'SOBBMDR12A8C13253B', 'SOBXHDL12A81C204C0', 'SOBYHAJ12A6701BF1D', 'SODACBL12A8C13C273', 'SODDNQT12A6D4F5F7E', 'SODXRTY12AB0180F3B', 'SOFGUAY12AB017B0A8', 'SOFRQTD12A81C233C0', 'SOHQWYZ12A6D4FA701', 'SOIYTOA12A6D4F9A23', 'SOIZAZL12A6701C53B', 'SOJNNUA12A8AE48C7A', 'SOJPFQG12A58A7833A', 'SOKRIMP12A6D4F5DA3', 'SOLLGNU12AF72A4D4F', 'SOMGIYR12AB0187973', 'SOMLMKI12A81C204BC', 'SOMSQJY12A8C138539', 'SONSAEZ12A8C138D7A', 'SOOKGRB12A8C13CD66', 'SOPCVQE12AC468AF36', 'SOQIVUD12AB01821D2', 'SOQJLDY12AAF3B456D', 'SOQLCKR12A81C22440', 'SORPMYJ12AF729EB90', 'SORQHCG12A58A7EEBA', 'SORUFVF12AB018230B', 'SORWLTW12A670208FA', 'SORZASF12A6D4F8CFA', 'SOSYBEV12AB0182933', 'SOTFATN12A6D4FA74D', 'SOTLVCL12AB0182D22', 'SOTRSFZ12A8C142BF6', 'SOUKXIN12A8C133C7F', 'SOVHRGF12A8C13852F', 'SOVQEYZ12A8C1379D8', 'SOVYIYI12A8C138D88', 'SOWQLXP12AF72A08A2', 'SOWSPUS12AC468BEE3', 'SOXMIUS12A8C13CD59', 'SOXRXDG12A8C131DE5', 'SOXZQDE12A8C135833', 'SOYHEPA12A8C13097F', 'SOZOBWN12A8C13

In [40]:
target_song_id = "SOBBMDR12A8C13253B"

In [41]:
def generate_recommended_ids():
    corr_mat = dict()
    user_of_target_song = all_users_of_song[target_song_id]
    
    for ind, song_id in tqdm(enumerate(all_songs)):
        user_by_song_id =  all_users_of_song[song_id]
        score = len(user_of_target_song.intersection(user_by_song_id))

        if score > 0:
            score = score/len(user_of_target_song.union(user_by_song_id))
        else:
            score = 0
        corr_mat[song_id] = score
        
    return sorted(corr_mat.items(), key=lambda item: item[1], reverse=True)[1:13]

In [42]:
corr_mat = generate_recommended_ids()

10000it [00:00, 42935.89it/s]


In [43]:
data = pd.DataFrame(corr_mat)
data

,0,1
0,SOENLMR12A58A78ADD,0.023411
1,SOZITSW12A6D4FB173,0.022876
2,SOOBUXN12AB01887FA,0.021739
3,SOTUJKY12AB018119F,0.020243
4,SOMJFXX12A8C13E9F1,0.019685
5,SOVIGZG12A6D4FB188,0.019337
6,SOLBJNF12A6D4F791D,0.019084
7,SOFBNMW12A8C13D372,0.018587
8,SOJNJGQ12A6D4F62BC,0.018519
9,SOWGDDU12AF72A03DA,0.018330


In [38]:
query = "SELECT title FROM song_data WHERE song_id in {}".format(data[0])
query

'SELECT title FROM song_data WHERE song_id in 0     SOENLMR12A58A78ADD\n1     SOZITSW12A6D4FB173\n2     SOOBUXN12AB01887FA\n3     SOTUJKY12AB018119F\n4     SOMJFXX12A8C13E9F1\n5     SOVIGZG12A6D4FB188\n6     SOLBJNF12A6D4F791D\n7     SOFBNMW12A8C13D372\n8     SOJNJGQ12A6D4F62BC\n9     SOWGDDU12AF72A03DA\n10    SOZKSGW12A6D4FBFE1\n11    SOZFNIO12A67AD8F43\nName: 0, dtype: object'